In [2]:
import networkx as nx
import numpy as np
import pandas as pd
from collections import defaultdict
import random
import json
import pickle as pkl
from torch_geometric.utils import k_hop_subgraph
from ogb.nodeproppred import PygNodePropPredDataset

In [4]:
dataset = PygNodePropPredDataset(name="ogbn-arxiv", root="../../dataset") 

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph = dataset[0]
print(graph)

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])


In [8]:
id2paper   = pd.read_csv("nodeidx2paperid.csv")
paper2text = pd.read_csv("titleabs.tsv", names=["paperid", "title", "abstract"], sep="\t")  
print(id2paper.head())
print(paper2text.head())
node_idx    = id2paper["node idx"].values.tolist()
paper_idx   = id2paper["paper id"].values.tolist()
node2paper  = {id:paper for id, paper in zip(node_idx, paper_idx)}

paper_idx   = paper2text["paperid"].values.tolist()
title       = paper2text["title"].values.tolist()
paper2title = {paper:title for paper, title in zip(paper_idx, title)}

paper_idx   = paper2text["paperid"].values.tolist()
abstract    = paper2text["abstract"].values.tolist()
paper2abs   = {paper:ab for paper, ab in zip(paper_idx, abstract)}

   node idx   paper id
0         0    9657784
1         1   39886162
2         2  116214155
3         3  121432379
4         4  231147053
   paperid                                              title  \
0   200971           ontology as a source for rule generation   
1   549074  a novel methodology for thermal analysis a 3 d...   
2   630234  spreadsheets on the move an evaluation of mobi...   
3   803423  multi view metric learning for multi view vide...   
4  1102481    big data analytics in future internet of things   

                                            abstract  
0  This paper discloses the potential of OWL (Web...  
1  The semiconductor industry is reaching a fasci...  
2  The power of mobile devices has increased dram...  
3  Traditional methods on video summarization are...  
4  Current research on Internet of Things (IoT) m...  


In [25]:
graph_arxiv = nx.Graph()
nodes = [("P" + str(idx), {"title": paper2title[node2paper[idx]]}) for idx in range(graph.num_nodes)]
graph_arxiv.add_nodes_from(nodes)
edges = list(zip(["P" + str(x) for x in graph.edge_index[0].numpy().tolist()], ["P" + str(x) for x in graph.edge_index[1].numpy().tolist()]))
graph_arxiv.add_edges_from(edges, relation="reference")

In [26]:
from tqdm import tqdm
cnt = 0
for idx, node in tqdm(enumerate(list(graph_arxiv.nodes(data=False))[:500])):
    nodesl = nx.bfs_successors(graph_arxiv, node, depth_limit=4)
    nodesl = [nodel[0] for nodel in list(nodesl)][:20]
    if len(nodesl) < 10:
        continue
    subgraph = nx.subgraph(graph_arxiv, nodesl)
    if subgraph.number_of_edges() > 50:
        continue
    nx.write_gml(subgraph, path="../../input_arxiv/GML/graph_"  + str(cnt) + ".gml")
    nx.write_graphml(subgraph, path="../../input_arxiv/GraphML/graph_"  + str(cnt) + ".graphml", named_key_ids=True, encoding='utf-8')
    nx.write_edgelist(subgraph, path="../../input_arxiv/EdgeList/graph_" + str(cnt) + ".edgelist", delimiter="\t", encoding="utf-8")
    nx.write_adjlist(subgraph, path="../../input_arxiv/AdjList/graph_"  + str(cnt) + ".adjlist", delimiter="\t", encoding="utf-8")
    cnt += 1

500it [00:47, 10.42it/s]
